[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb)

# Prompt Engineering

In this notebook we'll explore the fundamentals of prompt engineering. We'll start by installing library prerequisites.

In [2]:
!pip install langchain openai


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [3]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```

Let's try sending this to a GPT-3 model. We will use the LangChain library but you can also use the `openai` library directly. In both cases, you will need [an OpenAI API key](https://beta.openai.com/account/api-keys).

We initialize a `text-davinci-003` model like so:

In [4]:
from langchain_openai import ChatOpenAI

# initialize the models
openai = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-4" or other available models
    temperature=0.7
)

And make a generation from our prompt.

In [5]:
result = openai.invoke(prompt)
print(f"Response: {result.content}")


Response: Hugging Face's `transformers` library, OpenAI's `openai` library, and Cohere's `cohere` library offer LLMs.


We wouldn't typically know what the users prompt is beforehand. So rather than writing the full prompt, we create a `PromptTemplate` with a single input variable `query`.

In [6]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

Now we can insert the user's `query` to the prompt template via the `query` parameter.

In [7]:
print(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [8]:
response = openai.invoke(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)
print(f"Response: {response.content}")

Response: The libraries and model providers that offer LLMs are Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


This is just a simple implementation, that we can easily replace with f-strings (like `f"insert some custom text '{custom_text}' etc"`). But using LangChain's `PromptTemplate` object we're able to formalize the process, add multiple parameters, and build the prompts in an object-oriented way.

Yet, these are not the only benefits of using LangChains prompt tooling.

## Few Shot Prompt Templates

Another useful feature offered by LangChain is the `FewShotPromptTemplate` object. This is ideal for what we'd call *few-shot learning* using our prompts.

To give some context, the primary sources of "knowledge" for LLMs are:

* **Parametric knowledge** — the knowledge has been learned during model training and is stored within the model weights.

* **Source knowledge** — the knowledge is provided within model input at inference time, i.e. via the prompt.

The idea behind `FewShotPromptTemplate` is to provide few-shot training as **source knowledge**. To do this we add a few examples to our prompts that the model can read and then apply to our user's input.

## Few-shot Training

Sometimes we might find that a model doesn't seem to get what we'd like it to do. We can see this in the following example:

In [9]:
from IPython.display import Markdown

In [10]:
prompt = """
Create an FAQ in Markdown based on the following questions and answers:
Q1: What is your return policy?
A1: We accept returns within 30 days with receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries.
Q3: How can I track my order?
A3: Use the tracking link in your confirmation email.
"""

openai.temperature = 0.1  # increase creativity/randomness of output

Markdown(openai.invoke(prompt).content)

## Frequently Asked Questions

### Q: What is your return policy?
A: We accept returns within 30 days with receipt.

### Q: Do you ship internationally?
A: Yes, we ship to over 50 countries.

### Q: How can I track my order?
A: Use the tracking link in your confirmation email.

The output results are okay. However, what if the Markdown headers aren't at the correct level (wrong mumber of hashes), and what if we want more interesting formatting?

We could explain the desired formatting in painful detail in a large wordy paragraph, but that might be a bit fiddly and would likely lead to hallucinations. 

If a picture is worth a thousand words then examples are probably worth at least one hundred! We can give **examples** of what the outputs should look like - i.e. few-shot training.

In [11]:
prompt = """
Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you’ll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you’ll receive a **tracking link** via email.

---

Now generate the FAQ section for this data:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.

"""

Markdown(openai.invoke(prompt).content)

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa, Mastercard, PayPal, and Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

---

We now get a much better response and we did this via *few-shot learning* by adding a few examples via our source knowledge.

Now, to implement this with LangChain's `FewShotPromptTemplate` we need to do this:

In [12]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from IPython.display import Markdown
from langchain_openai import ChatOpenAI

# Create FAQ examples with both input_answers and formatted output_answers
examples = [
    {
        "question1": "What is your return policy?",
        "input_answer1": "We accept returns within 30 days with the original receipt.",
        "output_answer1": "We accept returns within **30 days** with the original **receipt**.",
        "question2": "Do you ship internationally?",
        "input_answer2": "Yes, we ship to over 50 countries worldwide.",
        "output_answer2": "Yes, we ship to over **50 countries** worldwide.",
        "question3": "How can I track my order?",
        "input_answer3": "After your order is shipped, you'll receive a tracking link via email.",
        "output_answer3": "After your order is shipped, you'll receive a **tracking link** via email."
    }
]

# Format for each example
example_template = """
Example input:
Q1: {question1}
A1: {input_answer1}
Q2: {question2}
A2: {input_answer2}
Q3: {question3}
A3: {input_answer3}

Example Output:

# Frequently Asked Questions

## [1. {question1}](#1-{question1})

{output_answer1}

## [2. {question2}](#2-{question2})

{output_answer2}

## [3. {question3}](#3-{question3})

{output_answer3}

---
"""

# Create prompt template for examples
example_prompt = PromptTemplate(
    input_variables=["question1", "input_answer1", "output_answer1", 
                    "question2", "input_answer2", "output_answer2", 
                    "question3", "input_answer3", "output_answer3"],
    template=example_template
)

# Instructions prefix
prefix = """Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**
"""

# User input format and instructions
suffix = """
Now generate the FAQ section for this data:
Q1: {question1}
A1: {input_answer1}
Q2: {question2}
A2: {input_answer2}
Q3: {question3}
A3: {input_answer3}
"""

# Create the few-shot prompt template
faq_few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question1", "input_answer1", 
                    "question2", "input_answer2", 
                    "question3", "input_answer3"],
    example_separator="\n"
)

# Format with new FAQ questions
formatted_prompt = faq_few_shot.format(
    question1="What payment methods do you accept?",
    input_answer1="We accept Visa, Mastercard, PayPal, and Apple Pay.",
    question2="Can I change my shipping address after ordering?",
    input_answer2="Only if your order hasn't shipped yet. Contact support ASAP.",
    question3="Do you offer gift wrapping?",
    input_answer3="Yes! You can select gift wrapping during checkout."
)

# Use with OpenAI
openai = ChatOpenAI(model_name="gpt-3.5-turbo")
response = openai.invoke(formatted_prompt)

# Display the formatted FAQ
Markdown(response.content)

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-What payment methods do you accept?)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-Can I change my shipping address after ordering?)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-Do you offer gift wrapping?)

Yes! You can select **gift wrapping** during checkout.

---

Now let's see what prompt this creates when we feed in a user query...

In [13]:
user_inputs = {
    "question1": "What payment methods do you accept?",
    "input_answer1": "We accept Visa, Mastercard, PayPal, and Apple Pay.",
    "question2": "Can I change my shipping address after ordering?",
    "input_answer2": "Only if your order hasn't shipped yet. Contact support ASAP.",
    "question3": "Do you offer gift wrapping?",
    "input_answer3": "Yes! You can select gift wrapping during checkout."
}

fromatted_prompt = faq_few_shot.format(**user_inputs)

print(fromatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**


Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-What is your return policy?)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-Do you ship internationally?)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-How can I track my order?)

After your order is shipped, you'll receive a **tracking link** via email.

---


Now generate the FAQ section for this data:
Q1: What paym

And to generate with this we just do:

In [14]:
resonse = openai.invoke(fromatted_prompt)

Markdown(response.content)

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-What payment methods do you accept?)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-Can I change my shipping address after ordering?)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-Do you offer gift wrapping?)

Yes! You can select **gift wrapping** during checkout.

---

Again, another good response.

However, this does seem somewhat convoluted. Why go through all of the above with `FewShotPromptTemplate`, the `examples` dictionary, etc — when we can do the same with a single f-string.

This approach is more robust and contains some nice features. One of those is the ability to include or exclude examples based on the length of our input FAQ data.

This is actually very important because the max length of our prompt and generation output is limited. This limitation is the *max context window*, and is simply the length of our prompt + length of our generation (which we define via `max_tokens`).

So we must try to maximize the number of example FAQs we give to the model as few-shot learning examples, while ensuring we don't exceed the maximum context window or increase processing times excessively.

Let's see how the dynamic inclusion/exclusion of examples works. First we need lots of FAQ examples:


In [43]:
# Expanded examples list with more FAQs
examples = [
    {
        "question1": "What is your return policy?",
        "input_answer1": "We accept returns within 30 days with the original receipt.",
        "output_answer1": "We accept returns within **30 days** with the original **receipt**.",
        "question2": "Do you ship internationally?",
        "input_answer2": "Yes, we ship to over 50 countries worldwide.",
        "output_answer2": "Yes, we ship to over **50 countries** worldwide.",
        "question3": "How can I track my order?",
        "input_answer3": "After your order is shipped, you'll receive a tracking link via email.",
        "output_answer3": "After your order is shipped, you'll receive a **tracking link** via email."
    },
    {
        "question1": "What payment methods do you accept?",
        "input_answer1": "We accept Visa, Mastercard, PayPal, and Apple Pay.",
        "output_answer1": "We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.",
        "question2": "Can I change my shipping address after ordering?",
        "input_answer2": "Only if your order hasn't shipped yet. Contact support ASAP.",
        "output_answer2": "Only if your order hasn't shipped yet. Contact support **ASAP**.",
        "question3": "Do you offer gift wrapping?",
        "input_answer3": "Yes! You can select gift wrapping during checkout.",
        "output_answer3": "Yes! You can select **gift wrapping** during checkout."
    },
    {
        "question1": "What are your store hours?",
        "input_answer1": "We're open Monday through Friday from 9am to 9pm, and weekends from 10am to 7pm.",
        "output_answer1": "We're open **Monday through Friday** from **9am to 9pm**, and **weekends** from **10am to 7pm**.",
        "question2": "Do you offer price matching?",
        "input_answer2": "Yes, we'll match any price from authorized retailers for identical products.",
        "output_answer2": "Yes, we'll match any price from **authorized retailers** for **identical products**.",
        "question3": "What is your warranty policy?",
        "input_answer3": "All electronics come with a standard 1-year manufacturer warranty.",
        "output_answer3": "All electronics come with a standard **1-year manufacturer warranty**."
    },
    {
        "question1": "How do I contact customer service?",
        "input_answer1": "You can reach us at support@example.com or call 555-123-4567.",
        "output_answer1": "You can reach us at **support@example.com** or call **555-123-4567**.",
        "question2": "Do you offer same-day delivery?",
        "input_answer2": "Yes, for orders placed before 2pm in selected metro areas.",
        "output_answer2": "Yes, for orders placed before **2pm** in selected **metro areas**.",
        "question3": "How do I cancel an order?",
        "input_answer3": "Log into your account and cancel within 1 hour of placing the order.",
        "output_answer3": "Log into your account and cancel within **1 hour** of placing the order."
    },
    {
        "question1": "Do you offer student discounts?",
        "input_answer1": "Yes, students with valid ID receive 15% off all purchases.",
        "output_answer1": "Yes, students with valid ID receive **15% off** all purchases.",
        "question2": "What is your privacy policy?",
        "input_answer2": "We never share your personal information with third parties without consent.",
        "output_answer2": "We **never share** your personal information with third parties without consent.",
        "question3": "Are your products environmentally friendly?",
        "input_answer3": "We use recyclable packaging and offer carbon-neutral shipping options.",
        "output_answer3": "We use **recyclable packaging** and offer **carbon-neutral shipping** options."
    },
    {
        "question1": "How do I apply for a refund?",
        "input_answer1": "Submit your request through our customer portal with your order number and reason.",
        "output_answer1": "Submit your request through our **customer portal** with your **order number** and reason.",
        "question2": "Do you have a loyalty program?",
        "input_answer2": "Yes! Earn 1 point for every dollar spent and redeem for discounts.",
        "output_answer2": "Yes! Earn **1 point** for every dollar spent and redeem for **discounts**.",
        "question3": "What are the system requirements?",
        "input_answer3": "Our software requires Windows 10/11 or macOS 10.15+, 8GB RAM, and 2GB storage.",
        "output_answer3": "Our software requires **Windows 10/11** or **macOS 10.15+**, **8GB RAM**, and **2GB storage**."
    },
    {
        "question1": "How long does shipping take?",
        "input_answer1": "Standard shipping takes 3-5 business days, and express shipping takes 1-2 business days.",
        "output_answer1": "Standard shipping takes **3-5 business days**, and express shipping takes **1-2 business days**.",
        "question2": "Do you have physical stores?",
        "input_answer2": "Yes, we have 12 locations across North America. Find the nearest one on our website.",
        "output_answer2": "Yes, we have **12 locations** across North America. Find the nearest one on our **website**.",
        "question3": "How do I reset my password?",
        "input_answer3": "Click 'Forgot Password' on the login page and follow the email instructions.",
        "output_answer3": "Click **'Forgot Password'** on the login page and follow the email instructions."
    },
    {
        "question1": "What's your wholesale policy?",
        "input_answer1": "For orders over $500, contact our wholesale department for special pricing.",
        "output_answer1": "For orders over **$500**, contact our **wholesale department** for special pricing.",
        "question2": "How can I become a vendor?",
        "input_answer2": "Fill out the vendor application form on our Partners page for consideration.",
        "output_answer2": "Fill out the **vendor application form** on our **Partners page** for consideration.",
        "question3": "Do you offer installation services?",
        "input_answer3": "Yes, professional installation is available for an additional fee in most areas.",
        "output_answer3": "Yes, **professional installation** is available for an additional fee in **most areas**."
    }
]

Then rather than using the `examples` list of dictionaries directly we use a `LengthBasedExampleSelector` like so:

In [44]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

Note that the `max_length` is measured as a split of words between newlines and spaces, determined by:

In [45]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


Then we use the selector to initialize a `dynamic_prompt_template`.

In [46]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

# First create the example selector with your FAQ examples
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=700  # adjust based on your context window needs
)

# Then create the dynamic prompt template
dynamic_faq_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question1", "input_answer1", 
                    "question2", "input_answer2", 
                    "question3", "input_answer3"],
    example_separator="\n"
)

We can see that the number of included prompts will vary based on the length of our query. First let's start with a relatively short user input.

In [47]:
# Using a dictionary of inputs
user_inputs = {
    "question1": "What payment methods do you accept?",
    "input_answer1": "We accept Visa, Mastercard, PayPal, and Apple Pay.",
    "question2": "Can I change my shipping address after ordering?",
    "input_answer2": "Only if your order hasn't shipped yet. Contact support ASAP.",
    "question3": "Do you offer gift wrapping?",
    "input_answer3": "Yes! You can select gift wrapping during checkout."
}

formatted_prompt = dynamic_faq_prompt.format(**user_inputs)

As you can see, the dynamically formatted prompt includes an number of examples.

In [48]:
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**


Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-What is your return policy?)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-Do you ship internationally?)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-How can I track my order?)

After your order is shipped, you'll receive a **tracking link** via email.

---


Example input:
Q1: What payment methods do you accept?
A1

Here is the response we get when we use this prompt with the LLM.

In [49]:
respond = openai.invoke(
    formatted_prompt
)
Markdown(respond.content)

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-What payment methods do you accept?)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-Can I change my shipping address after ordering?)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-Do you offer gift wrapping?)

Yes! You can select **gift wrapping** during checkout.

However, what if the user inputs are much longer, using more off the availabel context window?

In [51]:
# Using a dictionary of inputs with very long questions and answers
user_inputs = {
    "question1": "What are all the different payment methods that you accept for online purchases, including credit cards, digital wallets, bank transfers, and any special financing options that might be available for customers?",
    "input_answer1": "We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For credit cards, we accept Visa, Mastercard, American Express, and Discover. We also support digital wallet payments through PayPal, Apple Pay, Google Pay, Samsung Pay, and Amazon Pay. Additionally, we offer bank transfer options including ACH transfers, wire transfers, and direct debit for customers who prefer traditional banking methods. For larger purchases, we provide financing options through Affirm, Klarna, and our own in-house financing program with flexible payment plans ranging from 6 to 36 months. We also accept cryptocurrency payments including Bitcoin, Ethereum, and several other major cryptocurrencies for tech-savvy customers.",
    "question2": "Is it possible for me to modify or completely change my shipping address after I have already placed and confirmed my order, and if so, what are the specific conditions, timeframes, and procedures that I need to follow?",
    "input_answer2": "Yes, it is possible to modify your shipping address, but this depends entirely on the current status of your order in our fulfillment process. If your order has not yet been processed by our warehouse team and is still in 'pending' or 'confirmed' status, you can easily change the shipping address by logging into your account and accessing the order management section. However, once your order enters the 'processing' phase and our warehouse team begins preparing your items for shipment, address changes become much more complicated and may not be possible. If your order has already shipped, unfortunately we cannot redirect the package to a different address, but you can contact the shipping carrier directly to arrange for package interception or redirection services, though additional fees may apply. For the best chance of successful address modification, we strongly recommend contacting our customer support team immediately at support@company.com or calling our toll-free number.",
    "question3": "Do you provide gift wrapping services for the items that I purchase, and if you do, what are the different options available, what are the costs involved, and can I include personalized messages or special requests?",
    "input_answer3": "Absolutely! We offer comprehensive gift wrapping services to make your purchases extra special for any occasion. We have several gift wrapping options available: our standard gift wrap features elegant wrapping paper in various colors and patterns with matching ribbon and a bow for an additional $4.99 per item. Our premium gift wrap option includes luxury wrapping paper, silk ribbon, and decorative embellishments for $9.99 per item. For special occasions, we offer themed wrapping for holidays, birthdays, weddings, and baby showers at $7.99 per item. You can also add personalized gift messages up to 250 characters at no additional cost, and we'll include them on beautiful greeting cards. For an extra $2.99, you can upload custom messages or even photos to be printed on special cards. All gift-wrapped items are carefully packaged to ensure they arrive in perfect condition, and we offer discrete packaging options if you're sending gifts directly to recipients."
}

Noice that the number of examples used is reduced, as more of the context is taken up by the user input, and less remains for examples.

In [52]:
formatted_prompt = dynamic_faq_prompt.format(**user_inputs)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**


Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-What is your return policy?)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-Do you ship internationally?)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-How can I track my order?)

After your order is shipped, you'll receive a **tracking link** via email.

---


Now generate the FAQ section for this data:
Q1: What are 

The above shows that the `LengthBasedExampleSelector` class dynamically selects the examples without cutting any off halfway through. Now let's see what response this dynamically generated prompt gives.

In [53]:
respond = openai.invoke(
    formatted_prompt
)
Markdown(respond.content)

# Frequently Asked Questions

## [1. What payment methods do you accept for online purchases?](#1-What-payment-methods-do-you-accept-for-online-purchases)

We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For **credit cards**, we accept Visa, Mastercard, American Express, and Discover. We also support **digital wallet** payments through PayPal, Apple Pay, Google Pay, Samsung Pay, and Amazon Pay. Additionally, we offer **bank transfer** options including ACH transfers, wire transfers, and direct debit. For larger purchases, we provide financing options through Affirm, Klarna, and our own in-house financing program with flexible payment plans ranging from 6 to 36 months. We also accept **cryptocurrency payments** including Bitcoin, Ethereum, and other major cryptocurrencies.

## [2. Can I modify my shipping address after placing an order?](#2-Can-I-modify-my-shipping-address-after-placing-an-order)

Yes, it is possible to modify your shipping address depending on the current status of your order. If your order has not yet been processed by our warehouse team, you can change the shipping address by accessing the order management section in your account. Once the order enters the 'processing' phase, address changes may be more complicated. If your order has already shipped, contact the shipping carrier for services like package interception or redirection, though additional fees may apply. For address modifications, contact our customer support team immediately at support@company.com or call our toll-free number.

## [3. Do you offer gift wrapping services?](#3-Do-you-offer-gift-wrapping-services)

Absolutely! We offer comprehensive **gift wrapping** services for any occasion. Our options include standard gift wrap for $4.99 per item, premium gift wrap for $9.99 per item, and themed wrapping for holidays and special occasions at $7.99 per item. You can add personalized gift messages up to 250 characters at no additional cost, or opt for custom messages or photos on special cards for $2.99 extra. All gift-wrapped items are carefully packaged to ensure they arrive in perfect condition, with discrete packaging options available for direct gifts.

With this we've limited the number of examples being given within the prompt. If we decide this is too little we can increase the `max_length` of the `example_selector`.

In [56]:
dynamic_faq_prompt.example_selector.max_length = 1000
formatted_prompt = dynamic_faq_prompt.format(**user_inputs)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**


Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-What is your return policy?)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-Do you ship internationally?)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-How can I track my order?)

After your order is shipped, you'll receive a **tracking link** via email.

---


Example input:
Q1: What payment methods do you accept?
A1

These are just a few examples of the prompt tooling available in LangChain. For example, there is actually an entire other set of example selectors beyond the `LengthBasedExampleSelector`. We'll cover them in detail in upcoming notebooks, or you can read about them in the [LangChain docs](https://langchain-cn.readthedocs.io/en/latest/modules/prompts/example_selectors.html).